# Predição de Consumo de Combustível

Criação de modelos de aprendizado supervisionado para predizer o consumo de combustível de carros.

## Imports

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import PolynomialFeatures

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

import preprocessing as pre

## Pré-processamento

Aqui foi realizado o carregamento dos dados originais de um arquivo CSV.

In [4]:
original_df = pd.read_csv('data/car_data_original.csv')

Após isso, realizou-se a remoção das instâncias de carros elétricos pois continham muitos atributos faltantes e removeu-se os atributos de consumo em rodovia e cidade pois podem ser altamente colineares com a saída, simplificando demais o trabalho do preditor. Depois, separou-se as instâncias de treinamento/validação das instâncias de teste com o código abaixo.

In [ ]:
# Removendo os atributos colineares
no_colinear_df = pre.drop_atributes(original_df, ['highway_mpg', 'city_mpg'])

# Removendo as instâncias com NaN
no_nan_df = pre.remove_instances_with_nan(no_colinear_df)

Choosing the variables to be analyzed (X) and predicted (y)

In [7]:
df_filtered = original_df.drop(columns=["city_mpg", "highway_mpg"])
X = df_filtered.drop(columns=["combination_mpg"])
y = df_filtered["combination_mpg"]

Determinando o tamanho dos conjuntos de Teste e Validação

In [8]:
test_size = 0.1

Forcing at least one instance of every category to appear in the fossil_fuel dataset.

As in no_electric_cars, there isnt any strategy for dealing with non represented instances during the testing phase, we force every instance to appear at least once in the training case.

In [9]:
train_indices = set()
categorical_features = X.select_dtypes(include=['object', 'bool']).columns
for feature in categorical_features:
    unique_values = X[feature].unique()
    for value in unique_values:
        index = X[X[feature] == value].index[0]
        train_indices.add(index)

train_indices = list(train_indices)
X_train_mandatory = X.loc[train_indices]
y_train_mandatory = y.loc[train_indices]

X = X.drop(train_indices)
y = y.drop(train_indices)
        
test_size = test_size * (len(X) + len(X_train_mandatory)) / len(X)

Separando o conjunto de Teste antes de começar qualquer treinamento ou avaliação de modelos.